### Imports

In [1]:
import pandas as pd
import json
import os, sys
from tqdm import tqdm
import numpy as np
import polars as pl
from polars import col as c

sys.path.append("../")

### table creation

In [19]:
dir_path = "../data/parsed_dataframes"
file_name = "lab_records.parquet"
file_path = os.path.join(dir_path, file_name)
lab_records = pd.read_parquet(file_path)

In [20]:
for column in lab_records.columns:
    lab_records[column] = lab_records[column].replace(to_replace="nan", value=np.nan)
    lab_records[column] = pd.to_numeric(lab_records[column], errors="ignore")

/var/folders/v_/vqhfnf3s4sb7g6nfby7hc9sr0000gn/T/ipykernel_87876/1777949469.py:3: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  lab_records[column] = pd.to_numeric(lab_records[column], errors="ignore")


In [23]:
pd.DataFrame(lab_records.isna().sum(), columns=["count_nans"]).sort_values(
    "count_nans"
).head(10)

,count_nans
record,0
gender,0
birthday,0
lab_date,0
"Гемоглобин общий, массовая концентрация в крови",2819
Гематокрит крови методом автоматизированного подсчёта,2819
"Кислород общий, молярная концентрация в венозной крови",3177
"Билирубин общий, молярная концентрация в венозной крови",3177
"Диоксид углерода, парциальное давление в венозной крови",3316
"Хлориды, молярная концентрация в венозной крови",3316


In [22]:
(
    pl.from_pandas(lab_records)
    .with_columns(
        c("birthday").cast(pl.String).str.to_datetime("%Y%m%d").dt.date(),
        c("lab_date").cast(pl.String).str.to_datetime("%d.%m.%Y %H:%M").dt.date(),
    )
    .with_columns((c("lab_date").dt.year() - c("birthday").dt.year()).alias("age"))
    # ["Кристаллы оксалата кальция, обнаружение в моче методом компьютер-ассистированной микроскопии"]
    # .unique()
    # .null_count()
    .head(3)
)

record,gender,birthday,"Тропонин Т сердечный, обнаружение в крови",lab_date,"Белок, обнаружение в моче с помощью автоматизированного определения тест-полоской","Билирубин общий, обнаружение в моче с помощью автоматизированного определения тест-полоской",Водородный показатель (pH) мочи с помощью автоматизированного определения тест-полоской,"Глюкоза, обнаружение в моче","Кетоновые тела, обнаружение в моче с помощью автоматизированного определения тест-полоской","Лейкоциты, обнаружение в моче методом автоматизированного определения","Нитриты, обнаружение в моче с помощью автоматизированного определения тест-полоской",Относительная плотность мочи в тесте с водной нагрузкой,Прозрачность мочи,"Уробилиноген, массовая концентрация в моче с помощью тест-полоски",Цвет мочи,"Эритроциты, обнаружение в моче методом автоматизированного определения","Бикарбонат стандартный, молярная концентрация в венозной крови","Билирубин общий, молярная концентрация в венозной крови",Водородный показатель (pH) венозной крови,"Водородный показатель (pH) венозной крови, откорректированный по температуре пациента",Гематокрит крови методом автоматизированного подсчёта,"Гемоглобин общий, массовая концентрация в крови","Глюкоза, молярная концентрация в венозной крови","Диоксид углерода, парциальное давление в венозной крови","Избыток оснований истинный, расчетная молярная концентрация в венозной крови","Избыток оснований стандартный, расчетная молярная концентрация в венозной крови","Калий, молярная концентрация в венозной крови","Кальций ионизированный, молярная концентрация в венозной крови","Карбоксигемоглобин, массовая фракция в венозной крови","Кислород общий, молярная концентрация в венозной крови","Кислород, парциальное давление при 50% сатурации в венозной крови","Лактат, молярная концентрация в венозной крови","Метгемоглобин, массовая фракция в венозной крови","Насыщение гемоглобина кислородом, массовая фракция в венозной крови","Натрий, молярная концентрация в венозной крови","Оксигемоглобин, массовая фракция в венозной крови",…,"Избыток оснований стандартный, расчетная молярная концентрация в капиллярной крови","Калий, молярная концентрация в капиллярной крови","Кальций ионизированный, молярная концентрация в капиллярной крови","Карбоксигемоглобин, массовая фракция в капиллярной крови","Кислород общий, молярная концетрация в капиллярной крови","Кислород, парциальное давление в капиллярной крови","Кислород, парциальное давление при 50% сатурации в капиллярной крови","Лактат, молярная концентрация в капиллярной крови","Метгемоглобин, массовая фракция в капиллярной крови","Насыщение гемоглобина кислородом, массовая фракция в капиллярной крови","Натрий, молярная концентрация в капиллярной крови","Оксигемоглобин, массовая фракция в капиллярной крови","Хлориды, молярная концентрация в капиллярной крови","Ферритин, массовая концентрация в сыворотке или плазме крови",Агрегационная активность тромбоцитов,Молярная разница концентраций ионов натрия с гидрокарбонатом и хлором в артериальной крови,"Эритроциты, обнаружение в кале","D-димер, массовая концентрация в бедной тромбоцитами плазме в собственных единицах иммунологическим методом","Железосвязывающая способность общая, молярная концентрация в сыворотке или плазме крови","Цилиндры гиалиновые, количество в моче методом автоматизированного подсчёта","Бласты, относительное количество в крови методом ручного подсчёта","Дрожжевые клетки, количество в поле зрения в осадке мочи методом световой микроскопии c большим увеличением","Гамма-глутамилтрасфераза, каталитическая концентрация в сыворотке или плазме крови","Щелочная фосфатаза, каталитическая концентрация в сыворотке или плазме крови","Лютеинизирующий гормон, концентрация в условных единицах в сыворотке или плазме крови","Прогестерон, молярная концентрация в сыворотке или плазме крови","Фолликулостимулирующий гормон, концентрация в условных единицах в сыворотке или плазме крови","Эстрадиол, массовая концентрация в биологических жидкостях","Кальций ионизирован